# Schema Alignment

In [ ]:
!pip install pandas -q
!pip install pandas-profiling -q
!pip install numpy -q

## Phase 1: Dataset Manipulation

In [ ]:
import pandas as pd
import numpy as np

#### Funzioni Di Manipolazione

In [ ]:
def employees_range(employees):
    if (employees < 2):
        return "1"
    elif (employees >= 2 and employees <= 10):
        return "2 TO 10"
    elif (employees >= 11 and employees <= 50):
        return "11 TO 50"
    elif (employees >= 51 and employees <= 200):
        return "51 TO 200"
    elif (employees >= 201 and employees <= 500):
        return "201 TO 500"
    elif (employees >= 501 and employees <= 1000):
        return "501 TO 1,000"
    elif (employees >= 1001 and employees <= 5000):
        return "1,001 TO 5,000"
    elif (employees >= 5001 and employees <= 10000):
        return "5,001 TO 10,000"
    elif (employees > 10000):
        return "10,000+"
    elif (pd.isna(employees)):
        return pd.NA

In [ ]:
def change_marketcap(marketcap):
    if 'T' in marketcap:
        marketcap = marketcap.replace('.', '')
        return marketcap.replace('T', '')
    else:
        return marketcap.replace('B', '')

In [ ]:
def change_marketcap_low(marketcap):
    if 't' in marketcap:
        marketcap = marketcap.replace('.', '')
        return marketcap.replace('t', '')
    else:
        return marketcap.replace('b', '')

### Dataset Scarano Yahoo

In [ ]:
scarano_yahoo = pd.read_csv("sources/aziende/Scarano-companies-finance.yahoo.com-1.csv")
scarano_yahoo.tail(500)

In [ ]:
scarano_yahoo.dtypes

In [ ]:
scarano_yahoo.isnull().sum()

In [ ]:
scarano_yahoo = scarano_yahoo.drop('Unnamed: 0', 1)
scarano_yahoo = scarano_yahoo.drop('Unnamed: 0.1', 1)
scarano_yahoo = scarano_yahoo.drop('type', 1)
scarano_yahoo = scarano_yahoo.drop('source', 1)
scarano_yahoo['employees']= scarano_yahoo.apply(lambda x: employees_range(x['employees']), axis=1)
scarano_yahoo.head(10)

### Dataset Scarano Investing

In [ ]:
scarano_investing = pd.read_csv("sources/aziende/scarano_investing.csv")
scarano_investing.tail(500)

In [ ]:
scarano_investing.dtypes

In [ ]:
scarano_investing.isnull().sum()

In [ ]:
scarano_investing = scarano_investing.drop('Unnamed: 0', 1)
scarano_investing = scarano_investing.drop('Unnamed: 0.1', 1)
scarano_investing = scarano_investing.drop('type', 1)
scarano_investing = scarano_investing.drop('source', 1)
scarano_investing['employees'] = scarano_investing['employees'].replace('-', np.nan)
scarano_investing['employees'] = scarano_investing['employees'].replace('NaN', np.nan)
scarano_investing['employees'] = scarano_investing['employees'].astype('float').astype('Int64')
scarano_investing['employees']= scarano_investing.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
scarano_investing.head(10)

### Dataset AeA CompaniesMarket

In [ ]:
aeacm = pd.read_csv("sources/aziende/aea_companiesmarket.csv")
aeacm.head()

In [ ]:
aeacm.dtypes

In [ ]:
aeacm.isnull().sum()

In [ ]:
aeacm = aeacm.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
aeacm.head(10)

###### Togliamo Change (1 Year), Rank e Change (1 Day). Troppo difficile da generalizzare il ranking e i change potrebbero non essere aggiornati

In [ ]:
aeacm.rename(columns={'change (1 year)':'oneyear'}, inplace=True)
aeacm.rename(columns={'change (1 day)':'oneday'}, inplace=True)
aeacm = aeacm.drop('oneyear', 1)
aeacm = aeacm.drop('rank', 1)
aeacm = aeacm.drop('oneday', 1)


In [ ]:
aeacm.rename(columns={'categories':'industry'}, inplace=True)
aeacm.rename(columns={'share price':'shareprice'}, inplace=True)
aeacm.rename(columns={'country':'headquarter'}, inplace=True)
aeacm['shareprice'] = aeacm['shareprice'].str.replace('$', '')
aeacm['shareprice'] = aeacm['shareprice'].replace(',','', regex=True)
aeacm['marketcap'] = aeacm['marketcap'].astype('string')
aeacm['marketcap'] = aeacm.apply(lambda x: change_marketcap_low(x['marketcap']), axis=1)
aeacm['marketcap'] = aeacm['marketcap'].str.replace('$', '')
aeacm['marketcap'] = aeacm['marketcap'].replace('N/A', pd.NA)
aeacm['marketcap'] = aeacm['marketcap'].replace('n/a', pd.NA)
aeacm['shareprice'] = aeacm['shareprice'].replace('N/A', pd.NA)
aeacm['shareprice'] = aeacm['shareprice'].replace('n/a', pd.NA)
aeacm.head()

In [ ]:
aeacm.tail(300)

### Dataset AeA ValueToday

In [ ]:
aeavt = pd.read_csv("sources/aziende/aea_valuetoday.csv")
aeavt.head()

In [ ]:
aeavt.dtypes

In [ ]:
toDrop = ['rank in country (jan-2021)', 'employee count as on date', 'market value (jan-01-2021)', 'world rank (sep-01-2021)',
         'world rank (jan-01-2021)', 'world rank (jan-2020)', 'stock category', 'about company business', 'business sector',
         'isin code', 'chairman', 'annual results for year ending', 'company website', 'stock exchange', 'market value (jan 1st 2020)', 'annual revenue in usd']
aeavt = aeavt.drop(toDrop, axis=1)
aeavt.head()
aeavt.rename(columns={'number of employees':'employees'}, inplace=True)
aeavt.rename(columns={'company business':'industry'}, inplace=True)
aeavt.rename(columns={'annual net income in usd':'revenue'}, inplace=True)
aeavt.rename(columns={'headquarters country':'headquarter'}, inplace=True)
aeavt.rename(columns={'market cap (sep-01-2021)':'marketcap'}, inplace=True)
aeavt['employees'] = aeavt['employees'].str.replace(r'[^0-9]+', '')
aeavt['employees'] = aeavt['employees'].replace('NaN', np.nan)
aeavt['employees'] = aeavt['employees'].astype('float').astype('Int64')
aeavt['employees']= aeavt.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
aeavt['marketcap'] = aeavt['marketcap'].str.replace(r'[^\d.]+', '')
aeavt['revenue'] = aeavt['revenue'].str.replace(r',', '.')
aeavt['revenue'] = aeavt['revenue'].str.replace(r'[^\d.]+', '')
aeavt.head()

### Dataset HirsutePippo ValueToday

In [ ]:
hpvalue = pd.read_csv("sources/aziende/hp_aziende_2.csv")
hpvalue.head(25)

In [ ]:
hpvalue.dtypes

In [ ]:
hpvalue.isnull().sum()

In [ ]:
toDrop = ['World Rank (Sep-01-2021)', 'Annual Revenue in USD', 'Market Value (Jan-01-2021)', 'Stock Category',
         'Company Website', 'Stock Exchange']
hpvalue = hpvalue.drop(toDrop, axis=1)
hpvalue.head()

In [ ]:
hpvalue.rename(columns={'Name':'name'}, inplace=True)
hpvalue.rename(columns={'Annual Net Income in USD':'revenue'}, inplace=True)
hpvalue.rename(columns={'Company Business':'industry'}, inplace=True)
hpvalue.rename(columns={'Headquarters Country':'headquarter'}, inplace=True)
hpvalue.rename(columns={'Number of Employees':'employees'}, inplace=True)
hpvalue.rename(columns={'CEO':'ceo'}, inplace=True)
hpvalue['employees'] = hpvalue['employees'].str.replace(r'[^0-9]+', '')
hpvalue['employees'] = hpvalue['employees'].replace('NaN', np.nan)
hpvalue['employees'] = hpvalue['employees'].astype('float').astype('Int64')
hpvalue['employees']= hpvalue.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
hpvalue['revenue'] = hpvalue['revenue'].str.replace(r',', '.')
hpvalue['revenue'] = hpvalue['revenue'].str.replace(r'[^\d.]+', '')
hpvalue.head()

### Dataset HirsutePippo Horizon

In [ ]:
hphorizon = pd.read_csv("sources/aziende/hp_aziende.csv")
hphorizon.head(25)

In [ ]:
hphorizon.dtypes

In [ ]:
hphorizon.isnull().sum()

In [ ]:
toDrop = ['Type', 'Est. of Ownership', 'National ID', 'SIC Code']
hphorizon = hphorizon.drop(toDrop, axis=1)
hphorizon.head()

In [ ]:
hphorizon.rename(columns={'Name':'name'}, inplace=True)
hphorizon.rename(columns={'Address':'headquarter'}, inplace=True)
hphorizon.rename(columns={'Industry':'industry'}, inplace=True)
hphorizon.head()

### Dataset MMS FTEuropeansCompanies

In [ ]:
path = 'sources/aziende/ftEuropeanCompanies2018.csv'

with open(path, 'r', encoding='utf-8') as f:
    mmsft = pd.read_csv(path, sep=';|"', engine='python').dropna(how='all', axis=1)
mmsft.tail(10)

In [ ]:
toDrop = ['founded']
mmsft = mmsft.drop(toDrop, axis=1)

In [ ]:
mmsft.rename(columns={'company_name':'name'}, inplace=True)
mmsft.rename(columns={'sector':'industry'}, inplace=True)
mmsft.rename(columns={'country':'headquarter'}, inplace=True)
mmsft.rename(columns={'revenue_euros':'revenue'}, inplace=True)
mmsft['employees'] = mmsft['employees'].replace('NaN', np.nan)
mmsft['employees'] = mmsft['employees'].astype('Int64')
mmsft['employees']= mmsft.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
mmsft['revenue'] = mmsft['revenue'].str.replace(r'M', '')
mmsft.head()

### Dataset MMS Iseg

In [ ]:
path = 'sources/aziende/lsegEuropeanCompanies2018.csv'

with open(path, 'r', encoding='utf-8') as f:
    mmsiseg = pd.read_csv(path, sep=';|"|\t', engine='python',index_col = False).dropna(how='all', axis=1)
mmsiseg.head(10)

In [ ]:
mmsiseg.rename(columns={'company_name':'name'}, inplace=True)
mmsiseg.rename(columns={'sector':'industry'}, inplace=True)
mmsiseg.rename(columns={'country':'headquarter'}, inplace=True)
mmsiseg.rename(columns={'revenue_euros':'revenue'}, inplace=True)
mmsiseg['revenue'] = mmsiseg['revenue'].str[0:4]
mmsiseg['revenue'] = mmsiseg['revenue'].str.replace(r'[^\d.]+', '')
mmsiseg.head()

In [ ]:
mmsiseg.tail(300)

### Zero8 Trustpilot

In [ ]:
z8trustpilot = pd.read_json('sources/aziende/Zero8-trustpilot.com-v1.json', lines=True)
z8trustpilot.head()

In [ ]:
z8trustpilot.info()

In [ ]:
z8trustpilot.dtypes

In [ ]:
z8trustpilot.isnull().sum()

In [ ]:
toDrop = ['mail', 'info', 'phone', 'reviews_count', 'overall_rating', 'excellent', 'great', 'average', 'poor', 'bad']
z8trustpilot = z8trustpilot.drop(toDrop, axis=1)

In [ ]:
z8trustpilot.rename(columns={'address':'headquarter'}, inplace=True)
z8trustpilot.head()

### Zero8 Valuetoday

In [ ]:
z8valuetoday = pd.read_json('sources/aziende/Zero8-value.today-v1.json', lines=True)
z8valuetoday.head()

In [ ]:
toDrop = ['worldRank', 'founders', 'foundedYear', 'businessSector']
z8valuetoday = z8valuetoday.drop(toDrop, axis=1)
z8valuetoday.head()

In [ ]:
z8valuetoday.rename(columns={'companyName':'name'}, inplace=True)
z8valuetoday.rename(columns={'marketValue':'marketcap'}, inplace=True)
z8valuetoday.rename(columns={'annualRevenueUSD':'revenue'}, inplace=True)
z8valuetoday.rename(columns={'headquartersCountry':'headquarter'}, inplace=True)
z8valuetoday.rename(columns={'nEmployees':'employees'}, inplace=True)
z8valuetoday.rename(columns={'CEO':'ceo'}, inplace=True)
z8valuetoday['employees'] = z8valuetoday['employees'].str.replace(r'[^0-9]+', '')
z8valuetoday['employees'] = z8valuetoday['employees'].replace('NaN', np.nan)
z8valuetoday['employees'] = z8valuetoday['employees'].astype('float').astype('Int64')
z8valuetoday['employees']= z8valuetoday.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
z8valuetoday['marketcap'] = z8valuetoday['marketcap'].str.replace(r'[^\d.]+', '')
z8valuetoday['revenue'] = z8valuetoday['revenue'].str.replace(r',', '.')
z8valuetoday['revenue'] = z8valuetoday['revenue'].str.replace(r'[^\d.]+', '')
z8valuetoday.head()

In [ ]:
z8valuetoday.dtypes

### PolloEPatatine ValueToday

In [ ]:
import json

openfile=open('sources/aziende/polloepatatine-value.today-v1.json')
jsondata=json.load(openfile)
pepvalue=pd.DataFrame(jsondata)

openfile.close()

pepvalue.head()

In [ ]:
toDrop = ['rank', 'market value']
pepvalue = pepvalue.drop(toDrop, axis=1)
pepvalue.head()

In [ ]:
pepvalue.dtypes

In [ ]:
pepvalue.rename(columns={'business':'industry'}, inplace=True)
pepvalue.rename(columns={'annual revenue':'revenue'}, inplace=True)
pepvalue.rename(columns={'country':'headquarter'}, inplace=True)
pepvalue['employees'] = pepvalue['employees'].str.replace(r'[^0-9]+', '')
pepvalue['employees'] = pepvalue['employees'].replace('', np.nan)
pepvalue['employees'] = pepvalue['employees'].replace('NaN', np.nan)
pepvalue['employees'] = pepvalue['employees'].astype('float').astype('Int64')
pepvalue['employees']= pepvalue.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
pepvalue['marketcap'] = pepvalue['marketcap'].str.replace(r'[^\d.]+', '')
pepvalue['revenue'] = pepvalue['revenue'].str.replace(r',', '.')
pepvalue['revenue'] = pepvalue['revenue'].str.replace(r'[^\d.]+', '')
pepvalue['revenue'] = pepvalue['revenue'].replace('', pd.NA)
pepvalue['website'] = pepvalue['website'].replace('', pd.NA)
pepvalue['headquarter'] = pepvalue['headquarter'].replace('', pd.NA)
pepvalue.tail(300)

In [ ]:
pepvalue.head(300)

### PolloEPatatine CompaniesMarketCap

In [ ]:
import json

openfile=open('sources/aziende/polloepatatine-companiesmarketcap.com-1.json')
jsondata=json.load(openfile)
pepmarket=pd.DataFrame(jsondata)

openfile.close()

pepmarket.head()

In [ ]:
toDrop = ['code', 'rank', 'change(1day)', 'change(1year)', 'competitor']
pepmarket = pepmarket.drop(toDrop, axis=1)
pepmarket.head()

In [ ]:
pepmarket.rename(columns={'business':'industry'}, inplace=True)
pepmarket.rename(columns={'country':'headquarter'}, inplace=True)
pepmarket.rename(columns={'sharePrice':'shareprice'}, inplace=True)
pepmarket['shareprice'] = pepmarket['shareprice'].str.replace('$', '')
pepmarket['shareprice'] = pepmarket['shareprice'].replace(',','', regex=True)
pepmarket['marketcap'] = pepmarket['marketcap'].str.replace('$', '')
pepmarket['marketcap'] = pepmarket['marketcap'].astype('string')
pepmarket['marketcap'] = pepmarket.apply(lambda x: change_marketcap(x['marketcap']), axis=1)
pepmarket['marketcap'] = pepmarket['marketcap'].replace('N/A', pd.NA)
pepmarket['shareprice'] = pepmarket['shareprice'].replace('N/A', pd.NA)
pepmarket.head()

In [ ]:
pepmarket.tail(300)

### CrispyMcData KPopFandom

In [ ]:
kpopfandom = pd.read_csv("sources/aziende/kpopfandom-companies.csv")
kpopfandom.head(3)

In [ ]:
kpopfandom.dtypes

In [ ]:
kpopfandom.isnull().sum()

In [ ]:
kpopfandom.info()

In [ ]:
toDrop = ["type", "key_people", "active_group_links","active_group_names","distributor_links","distributor_names","former_group_links","former_group_names","founded","founders","hangul","key_people_links","link","other_names","parternship_links","parternship_names","socials","soloist_links","soloist_names","subsidiary_links","subsidiary_names","websites"]
kpopfandom = kpopfandom.drop(toDrop, axis=1)
kpopfandom.head()


In [ ]:
kpopfandom.rename(columns={'locations':'headquarter'}, inplace=True)
kpopfandom.head()

### CrispyMcData KPoppingFandom

In [ ]:
kpopping = pd.read_csv("sources/aziende/kpopping-companies.csv")
kpopping.head(3)

In [ ]:
kpopping.dtypes

In [ ]:
kpopping.isnull().sum()

In [ ]:
kpopping.info()

In [ ]:
toDrop = ["link", "revenue", "active_artist_links","active_artist_names","active_artist_partnership_periods","active_group_links","active_group_names","active_group_partnership_periods","active_groups","artists_count","former_artist_links","former_artist_names","former_artist_partnership_periods","former_group_links","former_group_names","former_group_partnership_periods","former_groups","founded","founders","groups_count","introduction","native_names","parent_company","parent_company_link","subsidiary_links","subsidiary_names"]
kpopping = kpopping.drop(toDrop, axis=1)
kpopping.head()


In [ ]:
kpopping.rename(columns={'CEO':'ceo'}, inplace=True)
kpopping.rename(columns={'hometown':'headquarter'}, inplace=True)
kpopping.rename(columns={'socials':'website'}, inplace=True)
kpopping['employees'] = kpopping['employees'].astype('Int64')
kpopping['employees']= kpopping.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
kpopping.head()

### Querynator Indeed

In [ ]:
import json

openfile=open('sources/aziende/indeed.json')
jsondata=json.load(openfile)
indeed=pd.DataFrame(jsondata)

openfile.close()

indeed.head()

In [ ]:
toDrop = ["id", "happinessScore", "reviewScore", "jobOffersCount"]
indeed = indeed.drop(toDrop, axis=1)
indeed.head()

In [ ]:
indeed.rename(columns={'headquarters':'headquarter'}, inplace=True)
indeed.rename(columns={'employeesSize':'employees'}, inplace=True)
indeed.rename(columns={'industryType':'industry'}, inplace=True)
indeed.rename(columns={'websiteList':'website'}, inplace=True)
indeed.head()

### Querynator Yelp

In [ ]:
import json

openfile=open('sources/aziende/companies.json')
jsondata=json.load(openfile)
companies=pd.DataFrame(jsondata)

openfile.close()

companies.head()

In [ ]:
toDrop = ["id", "starsCount", "reviewsCount", "mobileNumber", "workingHours"]
companies = companies.drop(toDrop, axis=1)
companies.head()

In [ ]:
companies.rename(columns={'type':'industry'}, inplace=True)
companies.rename(columns={'address':'headquarter'}, inplace=True)
companies.head()

## Nasti Companies

In [ ]:
nasticompanies = pd.read_json(path_or_buf='sources/aziende/nasti_companies_clutch.jsonl', lines=True)
nasticompanies.head()

In [ ]:
toDrop = ["id", "headquarter", "business_entity", "payment_legal_filings", "client_reviews", "description", "rating", "min_project_size", "avg_hourly_rate", "founded" ]
nasticompanies = nasticompanies.drop(toDrop, axis=1)
nasticompanies['employees'] = nasticompanies['employees'].str[0:2]
nasticompanies['employees'] = nasticompanies['employees'].str.replace(r'[^\d.]+', '')
nasticompanies['employees'] = nasticompanies['employees'].replace('', np.nan)
nasticompanies['employees'] = nasticompanies['employees'].astype('float').astype('Int64')
nasticompanies['employees'] = nasticompanies['employees'].replace(np.nan, pd.NA)
nasticompanies['employees']= nasticompanies.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)

In [ ]:
nasticompanies.tail(50)

## Phase 2: Dataset Concatenation

In [ ]:
companies.info()

In [ ]:
indeed.info()

In [ ]:
dataset = [indeed, companies, scarano_yahoo, scarano_investing, aeavt, aeacm, hpvalue, hphorizon, mmsft, mmsiseg, z8trustpilot, z8valuetoday, pepvalue, pepmarket, kpopfandom, kpopping]

for d in dataset:
    d.info()

In [ ]:
result = pd.concat([indeed, companies, scarano_yahoo, scarano_investing, aeavt, aeacm, hpvalue, hphorizon, mmsft, mmsiseg, z8trustpilot, z8valuetoday, pepvalue, pepmarket, kpopfandom, kpopping], sort=False)
result['employees'].replace('', pd.NA)
result['headquarter'].replace('', pd.NA)
result['industry'].replace('', pd.NA)
result['name'].replace('', pd.NA)
result['website'].replace('', pd.NA)
result['ticker'].replace('', pd.NA)
result['ceo'].replace('', pd.NA)
result['revenue'].replace('', pd.NA)
result['marketcap'].replace('', pd.NA)
result['shareprice'].replace('', pd.NA)

result['employees'].fillna(np.nan, inplace=True)
result['headquarter'].fillna(np.nan, inplace=True)
result['industry'].fillna(np.nan, inplace=True)
result['name'].fillna(np.nan, inplace=True)
result['website'].fillna(np.nan, inplace=True)
result['ticker'].fillna(np.nan, inplace=True)
result['ceo'].fillna(np.nan, inplace=True)
result['revenue'].fillna(np.nan, inplace=True)
result['marketcap'].fillna(np.nan, inplace=True)
result['shareprice'].fillna(np.nan, inplace=True)
result.rename(columns={'marketcap':'marketcap_M'}, inplace=True)
result.rename(columns={'revenue':'revenue_M'}, inplace=True)

In [ ]:
for columns in result.columns:
    result[columns] = result[columns].str.upper() 
result.tail(10000)



In [ ]:
result.info()

In [ ]:
result.to_csv("alignedSchemas/companiesAligned.csv")

In [ ]:
from pandas_profiling import ProfileReport 

In [ ]:
#profile=ProfileReport(result)
#profile.to_notebook_iframe()